In [1]:
%load_ext autoreload
%autoreload 2

from nlp_utils import PreProcessing
from nlp_utils import TopicModeling
from google.cloud import bigquery
import pandas as pd
import time

In [2]:
schema = ""
dataset = "yougraph_v2"
bq_client = bigquery.Client(project=schema)

tm = TopicModeling.TopicModeling()

## Processando Títulos

In [3]:
df_videos = pd.read_pickle("../dados/df_videos_processado.pkl")
df_videos["video_title"] = df_videos.apply(lambda x: x["video_title"].replace("&quot;",'"'), axis=1)
df_videos.head()

,video_id,channel_id,channel_name,video_title,video_desc,comment_count,dislike_count,favorite_count,view_count,like_count,video_duration,published_at,polarity,subjectivity,updated_at,texto_processado,bow
0,64kWfoig8Vs,UC-8Uff7i2h5qtIvjXJDJqYA,Minhas Plantas,"Como Cuidar de Orquídeas - Vanda, a orquídea g...","Curso online, vasos, adubos, livros e outros p...",108.0,152.0,0.0,469118.0,4253.0,PT4M5S,2012-05-17 18:41:56+00:00,NaN,NaN,2020-10-14 09:26:52+00:00,"[cuidar_orquideas, vanda, orquidea]","[(0, 1), (1, 1)]"
1,8EDLSOxHveI,UC-8Uff7i2h5qtIvjXJDJqYA,Minhas Plantas,Como Cuidar de Orquídeas - Quantas vezes regar...,A jardineira Carol Costa ensina um truque efic...,32.0,31.0,0.0,121113.0,2858.0,PT1M27S,2011-10-13 13:39:29+00:00,NaN,NaN,2020-10-14 09:26:52+00:00,"[cuidar_orquideas, regar, vaso]","[(0, 1), (2, 1), (3, 1)]"
2,8PD4y3-kVgM,UC-8Uff7i2h5qtIvjXJDJqYA,Minhas Plantas,Como Cuidar de Orquídeas — Regando a Vanda,No segundo vídeo da série especial sobre culti...,93.0,42.0,0.0,209628.0,1817.0,PT3M2S,2012-06-12 01:13:24+00:00,NaN,NaN,2020-10-14 09:26:52+00:00,"[cuidar_orquideas, regar, vanda]","[(0, 1), (2, 1)]"
3,8j3i0ERhcvQ,UC-8Uff7i2h5qtIvjXJDJqYA,Minhas Plantas,Como Cuidar de Orquídeas - Aprenda a tirar muda,"Curso online, vasos, adubos, livros e outros p...",229.0,142.0,0.0,500179.0,4010.0,PT3M21S,2011-11-17 20:26:55+00:00,NaN,NaN,2020-10-14 09:26:52+00:00,"[cuidar_orquideas, aprender, tirar, mudo]","[(0, 1), (4, 1), (5, 1), (6, 1)]"
4,CTrb2kjWg3k,UC-8Uff7i2h5qtIvjXJDJqYA,Minhas Plantas,Como Cuidar de Orquídeas - Acabe com os pulgões,A jardineira Carol Costa mostra os métodos mai...,89.0,115.0,0.0,199476.0,1382.0,PT2M38S,2011-10-31 16:40:54+00:00,NaN,NaN,2020-10-14 09:26:52+00:00,"[cuidar_orquideas, acabar, pulgoes]","[(0, 1), (7, 1)]"


### Pré Processamento Títulos:

In [4]:
#NLP
stop_word_list =   ["aqui", "gente", "oi", "então", "ai", "aí", "ufa", "coisa", "[Aplausos]", "[Música]", "né","tá", "vídeo","ne", "ta", "ha", "one", "at",
                    "casar", "coisa","canal", "tipo", "vezes", "mano", "meio", "ea", "eo", "só", "ae", "eis", "caraca", "caramba", "feat", "ft",
                    "graciele_lacerda",  "flavia_calino", "richard_rasmussen", "rezende_evil", "divo_depressao", "receitar_cris", "luisa_mell",
                    "leo_stronda", "divo_depressao", "estevar_mundo", "sal_flor", "tiago_fonseca", "graciele_lacerda", "organizar_frescura", "eduardo_bueno",
                    "debora_aladim", "paula_stephania", "dani_noce	", "tpm_ju", "prof_paulo", "meteoro_doc", "mariano_saad", "familia_brancoala", "osf_rafar",
                    "oliveira", "isabella_fiorentino", "pit_money", "luca_pit", "murilo_coutar", "bianca_andrade", "manual_mundo", "receitar_pai", "dani_noce", "thiago_venturo",
                    "danilo_gentili", "tato_fersoza", "teodoro_tato", "of_the", "like", "it", "house", "danilo"
                    ] #lista de stop words extendida

pp = PreProcessing.PreProcessing(stop_words_list=stop_word_list)

10:13:28 - Adicionanado novas stopwords.
10:13:28 -    Serão adcionada(s) 76 palavra(s)
10:13:28 -    Tamanho da lista de stopwords: 416
10:13:28 -    Tamanho da lista de stopwords: 481


In [6]:
start_time = time.time()
# lista_documentos = df_video_transcriptions["transcription_pt"].to_list()
lista_documentos = df_videos["video_title"].to_list()

allowed_postags = ["NOUN","VERB", "PROPN"] # filtro de postags
min_count=5  # Ngram - confiança
threshold=10 # Ngram - suporte
grau=4       # Ngram - quadrigrams
token_to_lower=True #Converter tudo para minusculo 
token_acentuacao=False #Manter Acentuação

corpus, modelo_ngram = pp.pipeline_pre_processamento(lista_documentos, 
                                allowed_postags=allowed_postags,
                                token_to_lower=token_to_lower, 
                                token_acentuacao=token_acentuacao,
                                n_gram_grau=grau,
                                n_gram_min_count=min_count,
                                n_gram_threshold=threshold,
                                stop_word_list=stop_word_list,
                                multithreading=False
                                )
corpus = pp.remover_stop_words(corpus)

print("-> %s Segundos" % (time.time() - start_time))

14:56:03 - Iniciando pipeline de pré processamento:
14:56:03 - SingleCore Selecionado!
14:56:03 - Removendo Urls:
14:56:03 - Filtrando Pos Tags
14:58:55 - Tokenizando
14:58:55 - Removendo Stop Words
14:58:55 -     -> Expandindo Stop Words
14:58:55 - Adicionanado novas stopwords.
14:58:55 -    Serão adcionada(s) 76 palavra(s)
14:58:55 -    Tamanho da lista de stopwords: 481
14:58:55 -    Tamanho da lista de stopwords: 481
14:58:56 - Lematizando
15:02:04 - Ngrams: Extraindo os Ngrams a partir de um modelo pré treinado.
15:02:06 - Filtrando tokens menores que 2
-> 362.7690622806549 Segundos


In [7]:
tamanho = 1

aux_corpus = []
for doc in corpus:
    aux_doc = []
    if doc is None:
        aux_corpus.append(aux_doc)
        continue    
    for token in doc:
        if len(token) > tamanho:
            aux_doc.append(token)
    aux_corpus.append(aux_doc)

corpus = aux_corpus

In [12]:
id2word = pp.montar_id2word(corpus)

print(len(id2word))
id2word = pp.filtrar_extremos(id2word=id2word)
bow_corpus = pp.montar_bow(corpus, id2word)
print(len(id2word))

19410
5899


In [13]:
path = "../dados/novo_id2word"
id2word.save(path)

In [14]:
df_videos["texto_processado"] = corpus 
df_videos["bow"] = bow_corpus
df_videos.head()

,video_id,channel_id,channel_name,video_title,video_desc,comment_count,dislike_count,favorite_count,view_count,like_count,video_duration,published_at,polarity,subjectivity,updated_at,texto_processado,bow
0,64kWfoig8Vs,UC-8Uff7i2h5qtIvjXJDJqYA,Minhas Plantas,"Como Cuidar de Orquídeas - Vanda, a orquídea g...","Curso online, vasos, adubos, livros e outros p...",108.0,152.0,0.0,469118.0,4253.0,PT4M5S,2012-05-17 18:41:56+00:00,NaN,NaN,2020-10-14 09:26:52+00:00,"[cuidar_orquideas, vanda, orquidea]","[(0, 1), (1, 1)]"
1,8EDLSOxHveI,UC-8Uff7i2h5qtIvjXJDJqYA,Minhas Plantas,Como Cuidar de Orquídeas - Quantas vezes regar...,A jardineira Carol Costa ensina um truque efic...,32.0,31.0,0.0,121113.0,2858.0,PT1M27S,2011-10-13 13:39:29+00:00,NaN,NaN,2020-10-14 09:26:52+00:00,"[cuidar_orquideas, regar, vaso]","[(0, 1), (2, 1), (3, 1)]"
2,8PD4y3-kVgM,UC-8Uff7i2h5qtIvjXJDJqYA,Minhas Plantas,Como Cuidar de Orquídeas — Regando a Vanda,No segundo vídeo da série especial sobre culti...,93.0,42.0,0.0,209628.0,1817.0,PT3M2S,2012-06-12 01:13:24+00:00,NaN,NaN,2020-10-14 09:26:52+00:00,"[cuidar_orquideas, regar, vanda]","[(0, 1), (2, 1)]"
3,8j3i0ERhcvQ,UC-8Uff7i2h5qtIvjXJDJqYA,Minhas Plantas,Como Cuidar de Orquídeas - Aprenda a tirar muda,"Curso online, vasos, adubos, livros e outros p...",229.0,142.0,0.0,500179.0,4010.0,PT3M21S,2011-11-17 20:26:55+00:00,NaN,NaN,2020-10-14 09:26:52+00:00,"[cuidar_orquideas, aprender, tirar, mudo]","[(0, 1), (4, 1), (5, 1), (6, 1)]"
4,CTrb2kjWg3k,UC-8Uff7i2h5qtIvjXJDJqYA,Minhas Plantas,Como Cuidar de Orquídeas - Acabe com os pulgões,A jardineira Carol Costa mostra os métodos mai...,89.0,115.0,0.0,199476.0,1382.0,PT2M38S,2011-10-31 16:40:54+00:00,NaN,NaN,2020-10-14 09:26:52+00:00,"[cuidar_orquideas, acabar, pulgoes]","[(0, 1), (7, 1)]"


In [ ]:
df_videos.to_pickle("../dados/df_videos_processado.pkl")

# Tag Modelo

In [15]:
tag_modelo = "v0"

In [16]:
dict_freq_quadrigram = pp.get_frequencia_ngrams(corpus)
dict_freq_quadrigram = {k: v for k, v in sorted(dict_freq_quadrigram.items(),reverse=True, key=lambda item: item[1])}
dict_to_df_ngram = {"palavra":list(dict_freq_quadrigram.keys()), "frequencia": list(dict_freq_quadrigram.values())}

df_ngram = pd.DataFrame(data=dict_to_df_ngram)
df_ngram["tag_modelo"] = tag_modelo
df_ngram.head()

,palavra,frequencia,tag_modelo
0,dia_dia,328,v0
1,analisar_review,221,v0
2,gameplay_portugues,203,v0
3,valer_peno,197,v0
4,dr_ana,196,v0


In [3]:

#Ngramns
table_name = "{dataset}.tb_ngrams"

bq_client.load_table_from_dataframe(
    df_ngram, table_name
)

#Texto Processado
table_name = "{dataset}.tb_video_processado"

bq_client.load_table_from_dataframe(
    df_videos[["video_id", "texto_processado"]], table_name
)

# PROCESSANDO AS TRANSCRIÇÕES

In [22]:
df_video_transcriptions = bq_client.query(f'''SELECT video_id, transcription_pt FROM `{schema}.{dataset}.video_transcriptions` WHERE transcription_type != "Erro"''').to_dataframe()
df_video_transcriptions.head()

,video_id,transcription_pt
0,3XLlu8GC2gg,Meu Deus do céu! Estamos aqui para ver histór...
1,Av11cr0Dz3I,"Sejam bem-vindos ao Nerdologia Eu sou Atila, ..."
2,BeQ704srg7M,"Olá pessoal, nós vamos conversar aqui \num po..."
3,BUFtdeILVwQ,"Eu coloquei tudo nessas sacolinhas, vamos ver..."
4,Cu-zFo34Mnc,"A Motorola lançou este mês o moto E6 Plus, se..."


In [23]:
start_time = time.time()
lista_documentos = df_video_transcriptions["transcription_pt"].to_list()

corpus, _ = pp.pipeline_pre_processamento(lista_documentos, 
                                allowed_postags=allowed_postags,
                                token_to_lower=token_to_lower, 
                                token_acentuacao=token_acentuacao,
                                stop_word_list=stop_word_list,
                                multithreading=False,
                                modelo_ngram= modelo_ngram
                                )
corpus = pp.remover_stop_words(corpus)

print("-> %s Segundos" % (time.time() - start_time))

12:22:58 - Iniciando pipeline de pré processamento:
12:22:58 - SingleCore Selecionado!
12:22:58 - Removendo Urls:
12:23:27 - Filtrando Pos Tags
15:04:03 - Tokenizando
15:04:51 - Removendo Stop Words
15:04:51 -     -> Expandindo Stop Words
15:04:51 - Adicionanado novas stopwords.
15:04:51 -    Serão adcionada(s) 76 palavra(s)
15:04:51 -    Tamanho da lista de stopwords: 481
15:04:51 -    Tamanho da lista de stopwords: 481
15:05:09 - Lematizando
15:47:49 - Ngrams: Gerando Ngrams
    -> Grau: 2
    -> Min Count: 5
    -> Threshold: 10
15:48:19 - Filtrando tokens menores que 2
-> 12341.166129112244 Segundos


In [24]:
bow_corpus = pp.montar_bow(corpus, id2word)

In [25]:
df_video_transcriptions["transcricao_processada"] = corpus 
df_video_transcriptions["bow"] = bow_corpus
df_video_transcriptions.head()

,video_id,transcription_pt,transcricao_processada,bow
0,3XLlu8GC2gg,Meu Deus do céu! Estamos aqui para ver histór...,"[deus, ceu, historiar, acontecer, olho, taxar_...","[(5, 1), (7, 1), (11, 2), (29, 3), (34, 2), (4..."
1,Av11cr0Dz3I,"Sejam bem-vindos ao Nerdologia Eu sou Atila, ...","[nerdologia, atila, biologo, pesquisador, fanb...","[(6, 1), (7, 1), (25, 7), (36, 1), (45, 1), (5..."
2,BeQ704srg7M,"Olá pessoal, nós vamos conversar aqui \num po...","[conversar, intestino, falar, intestino, video...","[(7, 5), (8, 3), (11, 1), (28, 3), (34, 1), (4..."
3,BUFtdeILVwQ,"Eu coloquei tudo nessas sacolinhas, vamos ver...","[colocar, sacolinhas, mostrar, esperar, termin...","[(6, 2), (8, 1), (12, 1), (34, 1), (35, 1), (4..."
4,Cu-zFo34Mnc,"A Motorola lançou este mês o moto E6 Plus, se...","[motorola, lancou, mes, moto_plus, smartphone,...","[(6, 1), (7, 1), (12, 1), (44, 1), (45, 1), (4..."


In [2]:
table_name = "{dataset}.tb_video_transcriptions_processado"

bq_client.load_table_from_dataframe(
    df_video_transcriptions[["video_id", "transcricao_processada"]], table_name
)

In [38]:
df_video_transcriptions.to_pickle("../dados/tb_video_transcriptions_processado.pkl")

In [28]:
lista_documentos_tokenizado = corpus
lista_dict_modelos =  tm.treinar_multiplos_modelos(vetor_corpus=bow_corpus,
                                                            id2word=id2word,
                                                            lista_documentos_tokenizado=lista_documentos_tokenizado,
                                                            start=2,
                                                            step=3,
                                                            limit=50)

18:30:19 - Start: 2
18:30:19 - limit: 50
18:30:19 - Step: 3
18:30:19 - Gerando novo modelo: K=2
18:31:25 - Gerando novo modelo: K=5
18:32:42 - Gerando novo modelo: K=8
18:34:20 - Gerando novo modelo: K=11
18:36:10 - Gerando novo modelo: K=14
18:38:26 - Gerando novo modelo: K=17
18:40:52 - Gerando novo modelo: K=20
18:43:30 - Gerando novo modelo: K=23
18:46:25 - Gerando novo modelo: K=26
18:49:37 - Gerando novo modelo: K=29
18:53:26 - Gerando novo modelo: K=32
18:57:33 - Gerando novo modelo: K=35
19:02:00 - Gerando novo modelo: K=38
19:06:37 - Gerando novo modelo: K=41
19:11:33 - Gerando novo modelo: K=44
19:17:06 - Gerando novo modelo: K=47


In [30]:
dict_melhor_modelo = tm.retornar_melhor_modelo()
melhor_modelo = dict_melhor_modelo["modelo"]
dict_melhor_modelo

{'modelo': <gensim.models.ldamodel.LdaModel at 0x7f9427633130>,
 'coerencia': 0.417746097661952,
 'num_topico': 47}

In [36]:
df_palavras = tm.retornar_top_key_words_agrupado(melhor_modelo)
df_palavras["tag_modelo"] = tag_modelo


#Palavras chave:
table_name = "{dataset}.tb_topicos_palavras"

bq_client.load_table_from_dataframe(
    df_palavras, table_name
)

df_palavras.head()

,topico_id,palavra,peso,tag_modelo
0,0,colocar,0.043115,v0
1,0,ficar,0.031303,v0
2,0,bolar,0.025549,v0
3,0,leite,0.023436,v0
4,0,musicar,0.021096,v0


In [34]:
df_analise_palavras, _ = tm.retornar_top_key_words(dict_melhor_modelo["modelo"])
df_analise_palavras.to_csv("top_palavras_47.csv")
df_analise_palavras.head()

,palavra_1,prob_1,palavra_2,prob_2,palavra_3,prob_3,palavra_4,prob_4,palavra_5,prob_5,...,prob_26,palavra_27,prob_27,palavra_28,prob_28,palavra_29,prob_29,palavra_30,prob_30,topico
topico_0,colocar,0.043115,ficar,0.031303,bolar,0.025549,leite,0.023436,musicar,0.021096,...,0.006488,brigadeiro,0.006348,geladeira,0.006340,tirar,0.006084,cortar,0.006067,0
topico_1,ficar,0.040612,colar,0.035947,colocar,0.029071,cortar,0.019939,passar,0.017774,...,0.005639,dobrar,0.005570,comecar,0.005549,decorar,0.005406,musicar,0.005232,1
topico_2,aplicativo,0.028078,video,0.021810,conseguir,0.014446,ficar,0.012483,site,0.011263,...,0.005999,contar,0.005850,receber,0.005846,baixar,0.005825,abrir,0.005671,2
topico_3,terra,0.031542,ano,0.023979,planeta,0.023608,descobrir,0.014137,existir,0.013704,...,0.005944,luzir,0.005638,sol,0.005534,contar,0.005432,pessoa,0.005312,3
topico_4,musicar,0.070013,olhar,0.032205,achar,0.019199,chegar,0.015581,ficar,0.014228,...,0.004260,acabar,0.004147,video,0.004075,mostrar,0.004028,carro,0.004026,4


In [37]:
path = "../dados/modelo_47_transc"
dict_melhor_modelo["modelo"].save(path)
modelo_ngram.save(f"{path}.ngram")